In [ ]:
import os
import gensim
import re

In [ ]:
model_path = '/media/sf_VBox_Shared/Arabic/wordembedding'
corpus_path = '/media/sf_VBox_Shared/Arabic/Fiqh/plaintext/'

In [ ]:
def normalize_arabic(text):
    # Remove non-arabic characters
    nonarab_chars = '[^\u0621-\u064A ]'
    text = re.sub(nonarab_chars, '', text)
    text = text.strip()
    return text

class CorpusArabic(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                words = [normalize_arabic(w) for w in line.split(' ')]
                words = [w for w in words if len(w)>0]
                if not len(words)==0:
                    yield words

## Finetune existing model

In [ ]:
model_original = gensim.models.Word2Vec.load(os.path.join(model_path, 'wikipedia_cbow_100'))

In [ ]:
corpus = CorpusArabic(corpus_path)
model_original.train(corpus, total_examples=model_original.corpus_count, epochs=5)

In [ ]:
model_original.save(os.path.join(model_path, 'wikipedia_cbow_100_finetuned'))

In [ ]:
# Save vectors
model_original.wv.save(os.path.join(model_path, 'wikipedia_cbow_100_finetuned_wv'))

## Train original model

In [ ]:
model_custom = gensim.models.word2vec.Word2Vec(
    iter=10,
    size=100, 
    window=5,
    sg=0)

In [ ]:
corpus = CorpusArabic(corpus_path)

In [ ]:
#model_custom_w.reset_from(model_original)
model_custom.build_vocab(corpus)

In [ ]:
model_custom.train(corpus, 
                   model_custom.epochs, 
                   epochs=model_custom.epochs,
                   total_examples=model_custom.corpus_count)

In [ ]:
fname = 'fiqh-i{}-s{}-w{}-sg{}'.format(
    model_custom.epochs, 
    model_custom.vector_size,
    model_custom.window,
    model_custom.sg)

In [ ]:
model_custom.save(os.path.join(model_path, fname))
model_original.wv.save(os.path.join(model_path, fname+'_wv'))